# Exercise Sheet \# 7 - Interfacing Python with SQL databases

In this exercises, you are asked to design python code which can interact with a SQLite database. We will use again the [SQL database](http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip) from Exercise Sheet \#6. 

## Exercise 1

Write a python script, which covers questions 1.5 to 1.9 from Exercise Sheet \#5, using the SQLalchemy API.

In [3]:
from sqlalchemy import Column, ForeignKey, Integer, String, Numeric
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

Base = declarative_base() 

class Artists(Base):
  __tablename__ = 'artists'
  # Each column is also a python instance attribute
  artistid = Column(Integer, primary_key=True)
  name     = Column(String(120), nullable=False)

class Albums(Base):
  __tablename__ = 'albums'
  albumid  = Column(Integer, primary_key=True)
  title    = Column(String(160), nullable=False)
  artistid = Column(Integer, ForeignKey('artists.artistid'))
  artists  = relationship(Artists)
    
class Genres(Base):
  __tablename__ = 'genres'
  genreid  = Column(Integer, primary_key=True)
  name     = Column(String(120))

class Tracks(Base):
  __tablename__ = 'tracks'
  trackid     = Column(Integer, primary_key=True)
  albumid     = Column(Integer, ForeignKey('albums.albumid'))
  genreid     = Column(Integer, ForeignKey('genres.genreid'))    
  name        = Column(String(200))
  mediatypeid = Column(Integer)
  composer    = Column(String(220))
  milliseconds= Column(Integer)      
  bytes       = Column(Integer)
  unitprice   = Column(Numeric)
  albums = relationship(Albums)
  genres = relationship(Genres)

# Create an engine that stores data locally 
# (and create the corresponding db file if it does not exists)
engine = create_engine('sqlite:///chinook.db')

# Bind the engine to the metadata of the Base 
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine, autoflush=False) #autoflush set to false to avoid errors when inserting
session   = DBSession()


from sqlalchemy import func #to import functions used within queries
# Note the functions doc is available at https://docs.sqlalchemy.org/en/latest/orm/tutorial.html#common-filter-operators

#1.5: SELECT albumId, count(Trackid) FROM tracks group by albumId;
print('\n############ Number of tracks per album ###########\n')
q1_5 = session.query(Tracks.albumid, func.count(Tracks.trackid)).group_by(Tracks.albumid)
# print result
for _aid, _count in q1_5.all():
    print('Album Id: {}, Nb tracks: {}'.format(_aid, _count))

#1.6: SELECT AVG(milliseconds) from tracks;
print('\n############ Average track duration ###########\n')
q1_6 = session.query(func.avg(Tracks.milliseconds))
for _length in q1_6.first():
    print('Average track duration (in milliseconds): ' + str(_length))
    
#1.7: INSERT INTO artists SELECT MAX(artistid) +1, 'Bono' FROM artists;
print('\n############ Adding new artist ###########\n')
q1_7 = Artists(artistid= int(session.query(func.max(Artists.artistid)).first()[0]) + 1, name='Bono')
session.add(q1_7)
#session.commit() #when uncommented will add Bono every time the code is executed!

#1.8: SELECT * FROM artists WHERE artistid NOT IN (SELECT DISTINCT artistid FROM albums);
print('\n############ Artists who have no registered album ###########\n')
q1_8 = session.query(Artists).filter(~Artists.artistid.in_([x[0] for x in session.query(Albums.artistid).all()]))
# Note that .all() returns tuples even if we only query artist ids!
for _artist in q1_8.all():
    print(_artist.artistid, _artist.name)

#1.9: DELETE FROM artists WHERE name LIKE 'N%';
print('\n############ Artists whose name starts with N ###########\n')
q1_9 = session.query(Artists).filter(Artists.name.like('N%'))
for _artists in q1_9.all():
    print(_artists.artistid, _artists.name)


C:\Users\Axelle\AppData\Local\Temp\ipykernel_8772\3084418804.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()



############ Number of tracks per album ###########



OperationalError: (sqlite3.OperationalError) no such table: tracks
[SQL: SELECT tracks.albumid AS tracks_albumid, count(tracks.trackid) AS count_1 
FROM tracks GROUP BY tracks.albumid]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [1]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.automap import automap_base

engine = create_engine('chinook.db')

Base = automap_base()
Base.prepare(engine, reflect=True)

Session = sessionmaker(bind=engine)
session = Session()

Track = Base.classes.tracks
Album = Base.classes.albums

num_tracks_per_album = session.query(albums.Title, func.count(tracks.TrackId)).\
                                join(tracks).\
                                group_by(albums.Title).\
                                order_by(albums.Title).all()

print(num_tracks_per_album)






ArgumentError: Could not parse SQLAlchemy URL from string 'chinook.db'

## Exercise 2
Extend your program so that it also covers questions 2.5, 2.6 and 2.8 from previous Exercise Sheet.

In [2]:
#2.5 SELECT a.title, count(t.name) FROM albums a, tracks t, artists ar WHERE a.albumid = t.albumid AND ar.artistid = a.artistid AND ar.name = 'Philip Glass Ensemble' GROUP BY a.title;
print('\n############ Albums and number of song by Philip Glass Ensemble ###########\n')
q2_5 = session.query(Albums.title, func.count(Tracks.name)).join(Tracks).join(Artists).filter(Artists.name == 'Philip Glass Ensemble')

for _albums in q2_5.all():
    print(_albums.title, _albums[1])
    
#2.6 SELECT a.name FROM artists a, albums al, tracks t WHERE a.artistid = al.artistid AND al.albumid = t.albumid AND t.milliseconds >= (SELECT MAX(milliseconds) FROM tracks);
print('\n############ Artists who play the longest song in the database ###########\n')
q2_6 = session.query(Artists.name).join(Albums).join(Tracks).filter(Tracks.milliseconds >= session.query(func.max(Tracks.milliseconds)))

for _ar in q2_6.all():
    print(_ar.name)


#2.8 SELECT g.name FROM genres g WHERE g.genreid NOT IN (SELECT t.genreid FROM tracks t);
print('\n############ Genres for which there is no song ###########\n')
q2_8 = session.query(Genres.name).filter(~Genres.genreid.in_([x[0] for x in session.query(Tracks.genreid).all()]))

for _names in q2_8.all():
    print(_names[0])


############ Albums and number of song by Philip Glass Ensemble ###########



NameError: name 'session' is not defined